In [35]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl
from azure.ai.ml import Input

import json
import pandas as pd

In [36]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


credential = InteractiveBrowserCredential()

In [37]:
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "98dcfdf0-c14b-4a76-9961-31fff2d85560"
    resource_group = "sales-forecast"
    workspace = "sales-forecast-ws1"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Found the config file in: .\config.json


In [38]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

In [39]:
output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location

In [40]:
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

,
Workspace,sales-forecast-ws1
Subscription ID,98dcfdf0-c14b-4a76-9961-31fff2d85560
Resource Group,sales-forecast
Location,eastus2


In [41]:
import shutil
import os
import yaml


def create_ml_table(csv_file, output, delimiter=",", encoding="ascii"):
    os.makedirs(output, exist_ok=True)
    fname = os.path.split(csv_file)[-1]
    mltable = {
        "paths": [{"file": f"./{fname}"}],
        "transformations": [
            {"read_delimited": {"delimiter": delimiter, "encoding": encoding}}
        ],
    }
    with open(os.path.join(output, "MLTable"), "w") as f:
        f.write(yaml.dump(mltable))
    shutil.copy(csv_file, os.path.join(output, fname))

In [42]:
create_ml_table("train.csv", "./data/training-mltable-folder")

In [43]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

In [44]:
from azure.ai.ml.entities import AmlCompute

cluster_name = "sales-forecast-compute2"
compute = ml_client.compute.get(cluster_name)

In [45]:
exp_name="sales_forecast_exp1"

forecasting_job = automl.forecasting(
    compute=cluster_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="sales",
    primary_metric="NormalizedRootMeanSquaredError",
    n_cross_validations="auto",
    enable_model_explainability=True,
)

# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=18,
    trial_timeout_minutes=5,
    enable_early_termination=True,
    max_concurrent_trials=1,
)

# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
    time_column_name="date",
    forecast_horizon=14,
    frequency="D",
    target_lags="auto",
    country_or_region_for_holidays="US",
)

# Training properties are optional
forecasting_job.set_training(blocked_training_algorithms=["ExtremeRandomTrees"])

In [46]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(forecasting_job)  # submit the job to the backend

print(f"Created job: {returned_job}")

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'C:\Users\Dinesh.Sreekanthan\PycharmProjects\azure-sales-forecast\data\training-mltable-folder' 'https://salesforecastw1977535163.blob.core.windows.net/azureml-blobstore-1b1d8e1e-3008-4fb5-b35b-eee7668847c2/LocalUpload/47e045c56ae46181ddb7106d4d7bf204/training-mltable-folder' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


Created job: compute: azureml:sales-forecast-compute2
creation_context:
  created_at: '2023-03-30T09:18:27.303705+00:00'
  created_by: Sreekanthan
  created_by_type: User
display_name: yellow_plow_7p0fhbk5rs
experiment_name: sales_forecast_exp1
forecasting:
  country_or_region_for_holidays: US
  feature_lags: none
  forecast_horizon: 14
  frequency: D
  seasonality: auto
  short_series_handling_config: auto
  target_aggregate_function: none
  target_lags: auto
  time_column_name: date
  use_stl: none
id: azureml:/subscriptions/98dcfdf0-c14b-4a76-9961-31fff2d85560/resourceGroups/sales-forecast/providers/Microsoft.MachineLearningServices/workspaces/sales-forecast-ws1/jobs/yellow_plow_7p0fhbk5rs
limits:
  enable_early_termination: true
  max_concurrent_trials: 1
  max_cores_per_trial: -1
  max_trials: 1000
  timeout_minutes: 18
  trial_timeout_minutes: 5
log_verbosity: info
n_cross_validations: auto
name: yellow_plow_7p0fhbk5rs
outputs: {}
primary_metric: normalized_root_mean_squared_erro

In [ ]:
ml_client.jobs.stream(returned_job.name)


RunId: yellow_plow_7p0fhbk5rs
Web View: https://ml.azure.com/runs/yellow_plow_7p0fhbk5rs?wsid=/subscriptions/98dcfdf0-c14b-4a76-9961-31fff2d85560/resourcegroups/sales-forecast/workspaces/sales-forecast-ws1
